In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

In [2]:
## loading the trained model
model=load_model('churn_model.h5')

## load the encoder and scaler
with open('labelencoder_gender.pkl','rb') as file:
    labelencoder_gender=pickle.load(file)

with open('ohe_geo.pkl','rb') as file:
    ohe_geo=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)


In [13]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [9]:
## one hot encoding 'Geography'
geo_encoded=ohe_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded=pd.DataFrame(geo_encoded,columns=ohe_geo.get_feature_names_out(['Geography']))
geo_encoded

d:\gen ai udemy\ann chrun classification\virt\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [14]:
input_data=pd.DataFrame([input_data])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [ ]:
## label encoder 'Gender'

input_data['Gender']=labelencoder_gender.transform(input_data['Gender'])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [ ]:
## concatinating the data

input_data=pd.concat([input_data.drop(['Geography'],axis=1),geo_encoded],axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [ ]:
## standardization with scalar

input_data=scaler.transform(input_data)

In [20]:
input_data

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [21]:
## predict churn
prediction=model.predict(input_data)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step


array([[0.02880982]], dtype=float32)

In [25]:
prediction_prob = prediction[0][0]


In [26]:
prediction_prob

np.float32(0.02880982)

In [27]:
if prediction_prob>0.5:
    print("customer will likely churn")
else:
    print("customer will not churn")

customer will not churn
